In [1]:
import sys
import os
import numpy as np
from pathlib import Path
sys.path.append(str(Path(os.getcwd()).parent))
np.set_printoptions(threshold=sys.maxsize, suppress=True)

In [2]:
from loader import Loader
from constant import FeatureType, FileType
from copy import deepcopy
from skimage.measure import compare_nrmse
from keras.models import Model
from keras.optimizers import RMSprop
from keras.layers import Input, LSTM, Dense, RepeatVector
import time
import argparse
import numpy as np
import tensorflow as tf
import hashlib
import json

In [3]:
# ====== Random Seed Initialization ====== #
seed = 1234
tf.random.set_seed(seed)

parser = argparse.ArgumentParser()
args = parser.parse_args("")
args.exp_name = "solar power prediction using weather features"

# ====== Data Loading ====== #
args.plant = 126
args.spot = 174
args.start_date = "20191210"#"20190820"
args.end_date = "20191220"#"20190830" #"20200809"
args.batch_size = 64
args.x_frames = 3
args.y_frames = 1

# ====== Model Capacity ===== #
args.hid_dim = 256

# ====== Optimizer & Training ====== #
args.optim = 'RMSprop'
args.activation = 'relu'
args.lss = 'MSE'
args.lr = 0.001
args.epochs = 256
args.early_stop = 30
args.evaluation = 'NRMSE'

# ====== Experiment Variable ====== #
name_var1 = 'features'
list_var1 = [FeatureType.SUNSHINE,
                 FeatureType.HUMIDITY,
                 FeatureType.WIND_SPEED,
                 FeatureType.VISIBILITY,
                 FeatureType.GROUND_TEMPERATURE,
                 FeatureType.WIND_DIRECTION,
                 FeatureType.STEAM_PRESSURE,
                 FeatureType.TEMPERATURE,
                 FeatureType.PRECIPITATION,
                 FeatureType.DEW_POINT_TEMPERATURE,
                 FeatureType.ATMOSPHERIC_PRESSURE]

for i in range(len(list_var1)):
    if i < 1:
        continue
    if i > 1:
        break
    sub_list = list_var1[:i + 1]

    setattr(args, name_var1, str(sub_list))
    print(args)

    setattr(args, name_var1, sub_list)
    loader = Loader(args)
    dataset = loader.get_dataset()
    X_train, y_train = dataset['train']
    X_val, y_val = dataset['val']
    X_test, y_test = dataset['test']
#     print("X_train")
#     print(X_train)
    print("y_train")
    print(y_train)
#     print("X_val")
#     print(X_val)
    print("y_val")
    print(y_val)
#     print("X_test")
#     print(X_test)
    print("y_test")
    print(y_test)
    
    

Namespace(activation='relu', batch_size=64, early_stop=30, end_date='20191220', epochs=256, evaluation='NRMSE', exp_name='solar power prediction using weather features', features="[<FeatureType.SUNSHINE: '일조(hr)'>, <FeatureType.HUMIDITY: '습도(%)'>]", hid_dim=256, lr=0.001, lss='MSE', optim='RMSprop', plant=126, spot=174, start_date='20191210', x_frames=3, y_frames=1)
y_train
[[0.         0.         0.         0.         0.         0.
  0.         0.01253482 0.17409471 0.4178273  0.75348189 0.78551532
  0.55292479 0.33286908 0.40668524 0.454039   0.05988858 0.
  0.         0.         0.         0.         0.         0.        ]
 [0.         0.         0.         0.         0.         0.
  0.         0.00278552 0.14902507 0.55431755 0.83147632 0.97910864
  0.92896936 0.95682451 0.78830084 0.48050139 0.05292479 0.
  0.         0.         0.         0.         0.         0.        ]
 [0.         0.         0.         0.         0.         0.
  0.         0.00696379 0.1810585  0.59888579 0.7